In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the image size and the number of classes
IMG_SIZE = 224
NUM_CLASSES = 4

# Load the images and labels
images = []
labels = []
for i, error_type in enumerate(['Functional', 'System_level', 'Usability_level', 'Security level']):
    folder_path = f'/content/sample_data/Dataset/{error_type}'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(i)

# Convert to numpy arrays and normalize the images
images = np.array(images) / 255.0
labels = np.array(labels)

# Split the data into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

import tensorflow as tf

# Load pre-trained VGG16 model without the top layers
base_model = VGG16(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

# Define the model
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    Dropout(0.5),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# Train the model
batch_size = 32
history = model.fit(datagen.flow(train_images, train_labels, batch_size=batch_size),
                    steps_per_epoch=len(train_images) // batch_size,
                    validation_data=(test_images, test_labels),
                    epochs=30)

model.save('bug_classifier.h5')


Epoch 1/30
4/4 [==============================] - 4s 401ms/step - loss: 1.5646 - accuracy: 0.2031 - val_loss: 1.3761 - val_accuracy: 0.2121
Epoch 2/30
4/4 [==============================] - 2s 419ms/step - loss: 1.4319 - accuracy: 0.2422 - val_loss: 1.3940 - val_accuracy: 0.2727
Epoch 3/30
4/4 [==============================] - 2s 403ms/step - loss: 1.4684 - accuracy: 0.2500 - val_loss: 1.3638 - val_accuracy: 0.3939
Epoch 4/30
4/4 [==============================] - 2s 402ms/step - loss: 1.3939 - accuracy: 0.2500 - val_loss: 1.3701 - val_accuracy: 0.3030
Epoch 5/30
4/4 [==============================] - 2s 471ms/step - loss: 1.3722 - accuracy: 0.3203 - val_loss: 1.3602 - val_accuracy: 0.3636
Epoch 6/30
4/4 [==============================] - 2s 426ms/step - loss: 1.3641 - accuracy: 0.3281 - val_loss: 1.3511 - val_accuracy: 0.4545
Epoch 7/30
4/4 [==============================] - 2s 373ms/step - loss: 1.3471 - accuracy: 0.3125 - val_loss: 1.3444 - val_accuracy: 0.3636
Epoch 8/30
4/4 [====

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_images, test_labels, batch_size=128)
print("test loss, test accuracy:", results)


Evaluate on test data
1/1 [==============================] - 5s 5s/step - loss: 1.2763 - accuracy: 0.4545
test loss, test accuracy: [1.276265263557434, 0.4545454680919647]
